<a href="https://colab.research.google.com/github/Phubordin/portfolio-project/blob/main/portfolio-project/P05-EDA-and-Comparison-of-NYC-Flights-Data-2013-vs-2023-with-R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ติดตั้ง Package nycflights

In [ ]:
install.packages(c("nycflights13", "nycflights23"))

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



# Load Library

In [ ]:
library(nycflights23)

head(flights)

In [ ]:
library(nycflights13)
head(flights)

In [ ]:
# Intro
library(tidyverse)
library(nycflights13)
library(nycflights23)
library(glue)
library(rnaturalearth)
library(sf)
library(ggplot2)
library(purrr)

?flights # file มีเเถว 336,766 ถ้าเราโหลด nycflights23 เวลารัน
# Flights จะเป็น data 2023 เเล้ว เหมือนมัน update

write.csv(flights, file = "flights.csv") # เขียนไฟล์เพื่อดูขนาดไฟล์
df <- read_csv("flights.csv") # อ่านไฟล์ที่ต้องใชเ _csv เพระาเวลารันมันไม่เป็น tibble ให้

## homework 16.25 - 22.30
## manipulate data (nycflights13)

## นี่คือ table ทั้งหมดที่ file Default (nycflights13)
flights # primary key -> 1 lines = 1 row = 1 เที่ยวบน = 1 การเดินทาง
airlines # primary key -> name_airlines
airports # primary key -> name_air
planes # primary key -> number_planes
weather # primary key ->origin_location_specific_time

# 1

In [ ]:
## Let's start HOMEWORK--------------------------------------------------------------

### 1. ต้องการอยากทราบว่า คนบินไปรัฐไหนบ้างแตกต่างกันอย่างไร---------------

# 1. โหลดข้อมูลเมืองจาก rnaturalearth
cities <- ne_download(scale = "large", type = "populated_places", returnclass = "sf")

# 2. แปลงตาราง airports ให้เป็น sf object
airport_coords <- airports %>%
  select(faa, name, lat, lon) %>%
  st_as_sf(coords = c("lon", "lat"), crs = 4326, remove = FALSE)

# 3. เชื่อมสนามบินกับเมืองใกล้เคียงที่สุด
matched_cities <- st_join(airport_coords, cities, join = st_nearest_feature)

colnames(matched_cities) <- tolower(colnames(matched_cities))
colnames(matched_cities)[2] <- "name_airport"


# 4. เลือกคอลัมน์ที่ต้องการเพื่อจะนำไป Join
state <- matched_cities %>%
  select(faa, adm1name, name_airport)

## 5. หาจำนวนเที่ยวบินที่ไปถึงที่สนามบินปลายทาง
n_flight_des <- flights %>%
  filter(if_all(everything(), ~ !is.na(.))) %>%
  select(dest) %>%
  group_by(dest) %>%
  summarise(n_flights = n()) %>%
  arrange(- n_flights)

## ไม่ต้องไปดูตรงอื่นเริ่มตรงนี้ !!!!
## 6. Join table เพื่อดูว่าสนามบินที่ว่าตั้งอยู่เมืองไหน และจัดการค่า NA
prep_n_flights <- n_flight_des %>%
  left_join(state, by = c("dest" = "faa")) %>%
  select(name_airport, dest, adm1name, n_flights) %>%
  # filter(if_any(everything(), is.na)) %>% # ไว้ test ว่าคอลัมน์ไหนมี NA
  mutate(
    adm1name = case_when(
      is.na(adm1name) & dest == "STT" ~ "U.S. Virgin Islands",
      is.na(adm1name) & dest == "BQN" ~ "Puerto Rico",
      is.na(adm1name) & dest == "SJU" ~ "Puerto Rico",
      is.na(adm1name) & dest == "PSE" ~ "Puerto Rico",
      TRUE ~ adm1name
    ),
    name_airport = case_when(
      is.na(name_airport) & dest == "STT" ~ "Cyril E. King Airport",
      is.na(name_airport) & dest == "BQN" ~ "Rafael Hernandez Airport",
      is.na(name_airport) & dest == "SJU" ~ "Luis Munoz Marin Interenational Airpot",
      is.na(name_airport) & dest == "PSE" ~ "Mercedita Airport",
      TRUE ~ name_airport
    )
  )
# view()

## 7. นับจำนวนเที่ยวบินที่บินไปแต่ละที่ ว่าปี 2013 สนามบินในนิวยอร์กบินไปไหนมากที่สุดเรียงจากน้อยไปมาก

summary_data1 <- prep_n_flights %>%
  select(adm1name, n_flights) %>%
  group_by(adm1name) %>%
  summarise(n_flights = sum(n_flights)) %>%
  arrange(- n_flights) %>%
  slice_head(n = 5)
#summarise(sum(n_flights))

## แถม
## ลบเเถวที่มี NA ออกให้หมด
## show เเถวที่มี NA ประกอบ
## ลอง make chart คร่าวๆ

flights %>%
  # filter(if_all(everything(), ~ !is.na(.)))
  # filter(if_any(everything(), is.na))


  summary_data1 %>%
  ggplot(aes(x = reorder(adm1name,  n_flights), y = n_flights)) +
  geom_col() +
  coord_flip()

## ตอบ Florida, California, North Carolina
## --------------- End 1 ------------------

# 2

In [ ]:
### 2. อยากทราบว่า ถ้าไป Florida จะไปช่วงเดือนที่คนไปบ่อยสุด

library(nycflights13)

flights %>%
  filter(if_all(everything(), ~ !is.na(.))) %>%
  left_join(state, by = c("dest" = "faa")) %>%
  mutate(
    adm1name = case_when(
      is.na(adm1name) & dest == "STT" ~ "U.S. Virgin Islands",
      is.na(adm1name) & dest == "BQN" ~ "Puerto Rico",
      is.na(adm1name) & dest == "SJU" ~ "Puerto Rico",
      is.na(adm1name) & dest == "PSE" ~ "Puerto Rico",
      TRUE ~ adm1name
    ),
    name_airport = case_when(
      is.na(name_airport) & dest == "STT" ~ "Cyril E. King Airport",
      is.na(name_airport) & dest == "BQN" ~ "Rafael Hernandez Airport",
      is.na(name_airport) & dest == "SJU" ~ "Luis Munoz Marin Interenational Airpot",
      is.na(name_airport) & dest == "PSE" ~ "Mercedita Airport",
      TRUE ~ name_airport
    )
  ) %>%
  filter(adm1name == "Florida") %>%
  mutate(month = month(time_hour, label = TRUE)) %>%
  group_by(month) %>%
  summarise(total_flights = n()) %>%
  ggplot(aes(x = month, y = total_flights, group = 1)) +
  geom_line(color = "blue", size = 1.2) +
  geom_point(color = "red", size = 3) +
  labs(
    title = "Number of Flights to Florida by Month",
    x = "Month",
    y = "Total Flights"
  ) +
  theme_minimal()

## ตอบ เดือน 3 12 4 1 ตามลำดับ
## --------------- End 2 ------------------

# 3

In [ ]:
## 3. สายการบินไหนดีขาไปที่มีโอกาส
# ไฟต์ดีเลย์น้อยขาออก + ถึงก่อนเวลา
# ใช้เวลาบินน้อยสุด + ระยะเวลาบินต่อเที่ยวบิน ใน 1 ปี (2013 อยู่เเล้ว)
# * เงื่อนไขคือ เราต้องมาถึงสนามบินอย่างน้อย 3 ชม.
# (check in, ทานข้าว, เผื่อเวลาฉุกเฉินเร่งด่วน ซึ่ง) และรีบไปปลายทางเร็วทีุ่ด


flights %>%
  filter(if_all(everything(), ~ !is.na(.))) %>%
  # select(carrier, dep_delay, arr_delay, air_time) %>%
  left_join(airlines, by = "carrier") %>%
  left_join(state, by = c("dest" = "faa")) %>%
  select(name, adm1name, dep_delay, arr_delay, air_time) %>%
  filter(adm1name == "Florida") %>%
  # filter(if_any(everything(), is.na)) %>%
  select(1, 3:5) %>%
  mutate(mint_expect = -(dep_delay + arr_delay)) %>%
  select(1, 4:5) %>%
  arrange(- mint_expect) %>%
  group_by(name) %>%
  summarise(mint_tra = sum(mint_expect),
            time = sum(air_time),
            n_flights = n()) %>%
  mutate(avg = time / n_flights) %>%
  select(name, mint_tra, avg) %>%
  arrange(-mint_tra, avg)



## Ans.>>  Envoy Air, Endeavor Air Inc., ExpressJet Airlines Inc.
## --------------- End 3 ------------------


# 4


In [ ]:
## 4. สภาพอากาศ ขาไปควร
## - เลือกไปสนามบินไหน (หลักๆ 3 ที่ EWR(NJ), JFK(NY), LGA(NY))
## - ช่วงเวลากี่โมง

## ส่วนนี้ไม่เกีย่วกับข้อ 4จนถึง....
cal_cor_by_col <- function(df) {
  # ตรวจสอบว่า dataframe ต้องมีอย่างน้อย 2 คอลัมน์ (คอลัมน์แรกเป็น character)
  if (ncol(df) < 2) {
    stop("DataFrame must have at least 2 columns: one character column and numeric columns.")
  }

  # สร้างค่าของ x ที่เป็น 1:nrow(df)
  x <- 1:nrow(df)

  # สร้างการคำนวณค่า correlation สำหรับแต่ละคอลัมน์ใน df ที่เป็น numeric
  correlation_results <- df %>%
    select(-1) %>%  # เอาคอลัมน์แรก (character) ออก
    map_dfc(~ cor(x, .))  # คำนวณ correlation สำหรับแต่ละคอลัมน์

  # ใส่ชื่อคอลัมน์ของผลลัพธ์
  colnames(correlation_results) <- colnames(df)[-1]

  return(correlation_results)
}

df <- weather %>%
  select_if(is.numeric) %>%
  bind_cols(origin = weather$origin) %>%
  select(origin, 5:13) %>%
  select(!wind_gust, !visib) %>%
  select(1:8) %>%
  mutate(row_number = row_number()) %>%
  replace_na(list(
    temp = 0,
    dewp = 0,
    humid = 0,
    wind_dir = 0,
    wind_speed = 0,
    pressure = 0
  )) %>%
  group_by(origin) %>%
  summarise(cor_temp_vs_dewp = cor(df$temp, df$dewp))
  cal_cor_by_col()
  # filter(if_any(everything(), is.na)) %>%
  # map(~ sum(is.na(.))) # เช็คว่าคอลัมน์ไหนมี NA

  ## ส่วนนี้ไม่เกีย่วกับข้อ 4จนถึง....ตรงนี้!!

library(nycflights13)
library(dplyr)
library(ggplot2)

# รวมข้อมูล flights และ weather
flights_weather <- flights %>%
  left_join(weather, by = c("origin", "year", "month", "day", "hour")) %>%
  filter(!is.na(precip)) # กรองเฉพาะข้อมูลที่มีค่า precip

# คำนวณโอกาสฝนตกในแต่ละสนามบินและช่วงเวลา
rain_analysis <- flights_weather %>%
  mutate(rain = ifelse(precip > 0, 1, 0)) %>%
  group_by(origin, month, hour) %>%
  summarise(
    total_flights = n(),
    rainy_flights = sum(rain),
    rain_prob = mean(rain)
  ) %>%
  arrange(rain_prob) # เรียงตามโอกาสฝนตกจากน้อยไปมาก

# แสดงกราฟโอกาสฝนตกตามสนามบินและช่วงเวลา
ggplot(rain_analysis, aes(x = hour, y = rain_prob, color = origin)) +
  geom_line(size = 1) +
  facet_wrap(~ month, scales = "free_x") +
  labs(
    title = "Rain Probability by Airport and Time of Day",
    x = "Hour",
    y = "Rain Probability (%)"
  ) +
  theme_minimal()
 ## Ans 9, 10, 4, 11 (4 เพราะ จากข้อ 2 คนนิยมไปช่วงเดือนนี้ Top 3 ของเดือน)
## --------------- End 4 Final ------------------